In [261]:
import torch
import torch.nn as nn
from torch.autograd import Variable


In [262]:
# var = Variable(torch.randn(100, 3, 84, 84))
# n_features = 64
# lc1 = nn.Conv2d(3, n_features, 4, 2, 1, bias=False)
# lc2 = nn.Conv2d(n_features, n_features * 2, 4, 2, 0, bias=False)
# lc3 = nn.Conv2d(n_features*2, n_features*4, 4,2,1,bias=False)
# lp1 = nn.MaxPool2d(2)
# ll1 = nn.Linear(int(n_features*4*5*5/640), 640)
# vl1 = lc1(var)
# print("vl1\n", vl1.size())
# vl2 = lc2(vl1)
# print("vl2\n", vl2.size())
# vl3 = lp1(vl2)
# print("vl3\n", vl3.size())
# vl4 = lc3(vl3)
# print("vl4\n", vl4.size())
# vl4 = vl4.view(-1, 640)
# print("vl4\n", vl4.size())
# vl5 = ll1(vl4)
# print("vl5\n", vl5.size())


In [279]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from torchvision import transforms, datasets
import torch.utils.data


CUDA = torch.cuda.is_available()


class MagnusNet(nn.Module):
    
    @staticmethod
    def no_dim_reduction_conv(in_channels, out_channels, padding=1, bias=False):
        return nn.Conv2d(in_channels, out_channels, 3, 1, padding, bias=bias)
    
    @staticmethod
    def half_dim_reduction_conv(in_channels, out_channels, padding=1, bias=False):
        return nn.Conv2d(in_channels, out_channels, 4, 2, padding, bias=bias)
    
    def __init__(self, n_features):
        super(MagnusNet, self).__init__()
        self.n_features = n_features
        self.convnet = nn.Sequential(
            MagnusNet.half_dim_reduction_conv(3, n_features),
            nn.BatchNorm2d(n_features),
            nn.LeakyReLU(0.1, True),
            
            MagnusNet.half_dim_reduction_conv(n_features, n_features * 2, 0),
            nn.MaxPool2d(2),
            nn.BatchNorm2d(n_features*2),
            nn.LeakyReLU(0.1, True),
            
            MagnusNet.half_dim_reduction_conv(n_features * 2, n_features * 4),
            nn.BatchNorm2d(n_features*4),
            nn.LeakyReLU(0.1, True),
            nn.Conv2d(n_features * 4, n_features * 4, 5),
        )
        self.linearnet = nn.Sequential(
            nn.Linear(n_features * 4, n_features),
            nn.ReLU(True),
            nn.Dropout(0.3),
            nn.Linear(n_features, 1),
            nn.Sigmoid(),
        )
    
    def forward(self, x):
        x = self.convnet(x)
        x = x.view(-1, self.n_features*4)
        x = self.linearnet(x)
        return x
    
    @staticmethod
    def net_instance(n_features):
        instance = MagnusNet(n_features)
        if CUDA:
            instance.cuda()
        return instance

batch_size = 3
n_epochs = 10
n_features = 64

dataset = datasets.ImageFolder(root='/opt/ProjectsPy/neurohack/neurodata/neuro-train',
                                   transform=transforms.Compose([
                                       transforms.ToTensor()
                                   ])
                                   )
dataloader = torch.utils.data.DataLoader(dataset, batch_size, shuffle=True)

testset = datasets.ImageFolder(root='/opt/ProjectsPy/neurohack/neurodata/neuro-test',
                                   transform=transforms.Compose([
                                       transforms.ToTensor()
                                   ])
                                   )

testloader = torch.utils.data.DataLoader(testset, batch_size, shuffle=True)

model = MagnusNet.net_instance(n_features)

learning_rate = 0.01
beta_one = 0.81
beta_two = 0.999
optimizer = optim.Adam(model.parameters(), learning_rate, (beta_one, beta_two))

criterion = nn.MSELoss()


def train(epoch):
    model.train()
    for i, (x, label) in enumerate(dataloader):
        label = label.float()
        if CUDA:
            x, label = x.cuda(), label.cuda()
        x, label = Variable(x), Variable(label)
        model.zero_grad()
        output = model(x)
        loss = criterion(output, label)
        loss.backward()
        optimizer.step()
        print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
            epoch, i * len(x), len(dataloader.dataset),
                   100. * i / len(dataloader), loss.data[0]))
            
def test():
    model.eval()
    test_loss = 0
    correct = 0
    total = 0
    for i, (x, label) in enumerate(testloader):
        int_label = torch.LongTensor(label)
        label = label.float()
        if CUDA:
            x, label = x.cuda(), label.cuda()
        x, label = Variable(x, volatile=True), Variable(label)
        output = model(x)
        test_loss += criterion(output, label)
        output_tensor = output.data
        output_tensor = output_tensor.apply_(lambda x: 0.0 if x < 0.5 else 1.0)
        onp = output_tensor.numpy().flatten()
        lnp = int_label.numpy()
        correct += (onp == lnp).sum()
        total += int_label.size()[0]
    test_loss = test_loss.data.cpu().numpy()[0] / total
    
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{}\n'.format(
        test_loss, correct, total))

In [280]:
convnet = nn.Sequential(
            MagnusNet.half_dim_reduction_conv(3, n_features),
            nn.BatchNorm2d(n_features),
            nn.LeakyReLU(0.1, True),
            
            MagnusNet.half_dim_reduction_conv(n_features, n_features * 2, 0),
            nn.MaxPool2d(2),
            nn.BatchNorm2d(n_features*2),
            nn.LeakyReLU(0.1, True),
            
            MagnusNet.half_dim_reduction_conv(n_features * 2, n_features * 4),
            nn.BatchNorm2d(n_features*4),
            nn.LeakyReLU(0.1, True),
            nn.Conv2d(n_features * 4, n_features * 4, 5),
        )

In [281]:
linearnet = nn.Sequential(
            nn.Linear(n_features*4, n_features),
            nn.ReLU(True),
            nn.Dropout(0.3),
            nn.Linear(n_features, 1),
            nn.Sigmoid(),
        )

In [282]:
x = Variable(torch.randn(3, 3, 84, 84))
co = convnet(x)
print('co', co.size())
co = co.view(-1, n_features*4)
print('co', co.size())
lo = linearnet(co)
print('lo', lo.size())


co torch.Size([3, 256, 1, 1])
co torch.Size([3, 256])
lo torch.Size([3, 1])


In [283]:
test()


Test set: Average loss: 0.0993, Accuracy: 5/10

